In [99]:
import re
import numpy as np
import heapq
from functools import reduce,cache
from operator import mul
from bisect import insort
from itertools import permutations
from collections import deque,defaultdict
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
1
2
3
2024
"""

test_res = """\
15887950
16495136
527345
704524
1553684
12683156
11100544
12249484
7753432
5908254
"""

### Part 1
The most difficult part about this puzzle is that my cat uncharacteristically jumped on top of me and started napping, leaving me only capable of using like half of the keyboard. but I got to think about the question and changed all operations to bitwise to possibly save on titme. and then I just got to coding once I was released from cat jail!

R = S\*64, 64 = 2\*\*6\
R = R xor S\
R = R % 16777216\
R = S\/\/32, 32 = 2\*\*5\
R = R xor S\
R = R % 16777216\
R \* 2048, 2048 == 2\*\*11\
R = R xor S\
R = R % 16777216\


In [119]:
class SecretNumber:
    def __init__(self,txt):
        self.initials = list(map(int,txt.splitlines()))
        self.sequences = defaultdict(lambda:0)

    def calculate_n_times(self,s,n):
        for i in range(n):
            s = res.calculate(s)
        return s
    
    def find_seqs(self,n):
        # calculating the iterations and then the deltas
        deltases = []
        for s in res.initials:
            xs,ds= [s%10],[]
            for i in range(n):
                s = res.calculate(s)
                xs.append(s%10)
            # self.deltas.apend(xs)
            for x,y in zip(xs,xs[1:]):
                ds.append((y-x,y)) # delta, value
            deltases.append(ds)
        # print(deltases)

        # then, for each list of deltas, check 4-part sequences
        for deltas in deltases:
            seen = defaultdict(lambda:0) # seqs dict for each deltas
            for i, d in enumerate(deltas):
                if 0 < i-3 and d[1] > 0: # the 4 digit seq is in bounds
                    seq_tups = deltas[i-3:i+1]
                    seq = []
                    for tup in seq_tups:
                        seq.append(tup[0])
                    seqstr = str(seq) # a bit convoluted way of getting the right seq
                    # print(seqstr)
                    if seen[seqstr] == 0: # only add to global dict if not seen
                        self.sequences[seqstr] += d[1]
                    seen[seqstr] = 1

    def calculate(self,s):
        # R = S*64, 64 = 2**6
        r = s << 6
        assert r == s*64
        # R = R xor S
        s = self.mix(r,s)
        # R = R % 16777216
        s = self.prune(s)

        # R = S//32, 32 = 2**5
        r = s >> 5
        # R = R xor S
        s = self.mix(r,s)
        # R = R % 16777216
        s = self.prune(s)

        # R * 2048, 2048 == 2**11
        r = s << 11
        # R = R xor S
        s = self.mix(r,s)
        # R = R % 16777216
        s = self.prune(s)
        return s

    def mix(self, n, s):
        return n ^ s # 
    
    def prune(self, n):
        return n&(16777215) # == n%16777216

res = SecretNumber(input)

# res.calculate_changes_n_times()

# test 1
# test_results = list(map(int,test_res.splitlines()))
# s = 123
# for i in range(10):
#     s = res.calculate(s)
#     # print(s)
#     assert test_results[i] == s

# test 2
# print (f"test: {sum([res.calculate_changes_n_times(num,2000) for num in res.initials])}")
# assert total == 37327623

print (f"part 1: {sum([res.calculate_n_times(num,2000) for num in res.initials])}") # 3.3s
res.find_seqs(2000)
print(f"part 1: {max(res.sequences.values())}") # 12.8s



part 1: 16953639210
part 1: 1863


### Part 2 & Takeaways
- The biggest hiccup today was reading comprehension. The logic was nothing hard, but it took me 5 tries to really get what I had to do for part 2. Shout out to the folks who also tripped over the terms for helping me catch mine.
- The following is my code compiled and some playground testing for part 1.

In [ ]:
            # print(i_sequences)
            # if i_sequences[seqstr] > 0:
            #     self.sequences[seqstr] += i_sequences[seqstr]
            #     continue
            # if d > 0 :
            #     # print(i_sequences[str(seq)])
            #     # otherwise record locally
            #     i_sequences[seqstr] = d
            #     # print("     ",i_sequences[str(seq)])
            # print(i_sequences)

            # # attempt 4
            # max_d = max(deltas)
            # # indexes = []
            # for i, d in enumerate(deltas):
            #     if d == max_d:
            #         # indexes.append(i)
            #         seq_tups = deltas[i-3:i+1]
            #         seq = []
            #         for tup in seq_tups:
            #             seq.append(tup[0])
            #         seqstr = str(seq)
            #         print(seqstr)
            #         self.sequences[seqstr] += max_d[1]

            # # attempt 3
            # max_i,max_d = 0,(0,0)
            # for i, d in enumerate(deltas): 
            #     if max_d[1] < d[1]:
            #         max_i = i
            #         max_d = d

            # seq_tups = deltas[max_i-3:max_i+1]
            # seq = []
            # for tup in seq_tups:
            #     seq.append(tup[0])
            # seqstr = str(seq)
            # self.sequences[seqstr] += max_d[1]
            
            # # attempt 2
            # i_sequences = defaultdict(lambda:0) # seqs dict for each deltas
            # for i, d in enumerate(deltas):
            #     if i+4 < len(deltas) and d[1] > 0:
            #         d2 = deltas[i+4]
            #         if d[1] == d2[1]:
            #             seq_tups = deltas[i+1:i+5]
            #             seq = []
            #             for tup in seq_tups:
            #                 seq.append(tup[0])
            #             seqstr = str(seq)
            #             self.sequences[seqstr] += i_sequences[seqstr]

            # # attempt 1
            # # if it's been seen before, load it into global dict
            # seq = deltas[i+1:i+5]
            # seqstr = str(seq)
            # # print(i_sequences[seqstr])
            # if i_sequences[seqstr] > 0:
            #     self.sequences[seqstr] += i_sequences[seqstr]
            #     continue
            # if d > 0 :
            #     # print(i_sequences[str(seq)])
            #     # otherwise record locally
            #     i_sequences[seqstr] = d
            #     # print("     ",i_sequences[str(seq)])
            # print(i_sequences)
                


In [93]:
16777216/(2**24)

6-(6>>2)

print(100000000%16777216)
print(100000000&(16777216-1)) # which equals 100000000%16777216


print(45530%(2**4))
print(45530&(2**4-1))

print(2**6)
print(17*64)
print(17<<6)

print(2**5)
print(36//32)
print(36>>5)

print(2**11)
print(17*2048)
print(17<<11)

print(17%10)

xs = [0,1,2,3,4,5,6,7,8,9]
n = 6
xs[n-3:n+1]

16113920
16113920
10
10
64
1088
1088
32
1
1
2048
34816
34816
7


[3, 4, 5, 6]